In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
from scipy import stats
from scipy.stats import linregress
from config import weather_api_key
#from config import g_key
from citipy import citipy

In [2]:
#set "cit center" for Houston
Houston_lat = 29.756097853207656
Houston_lng = -95.3669907972107


#calculate greater Houston area by expanding lat and lng
lat = np.random.uniform(Houston_lat-1, Houston_lat+2, size=6500)
lng = np.random.uniform(Houston_lng-5, Houston_lng+2, size=6500)
city = []

#find nearest city using lat and lng
for x in range(len(lat)):
    city.append(citipy.nearest_city(lat[x], lng[x]).city_name)


In [36]:
#turn cities into df
df_city = pd.DataFrame({
    "City":city,
    "lat":lat,
    "lng":lng,
})

In [38]:
df_city.head(20)

,City,lat,lng
0,port arthur,28.854652,-93.769056
1,brownwood,31.157402,-99.100542
2,wharton,29.655389,-96.296697
3,el campo,29.269464,-96.287770
4,el campo,29.608325,-96.686032
5,taylor,30.631240,-97.064669
6,pleasanton,28.899829,-98.791165
7,copperas cove,31.341957,-98.399390
8,brownwood,31.732214,-98.453478
9,taylor,30.661726,-97.328933


In [5]:
#check length of cities
len(list(df["City"].unique()))

111

In [6]:
cities = list(df["City"].unique())
target_cities=[]

for i in cities:
    c=str(i).replace(" ","+")
    target_cities.append(c)

target_cities

['port+arthur',
 'brownwood',
 'wharton',
 'el+campo',
 'taylor',
 'pleasanton',
 'copperas+cove',
 'lufkin',
 'temple',
 'galveston',
 'uvalde',
 'lockhart',
 'brenham',
 'gatesville',
 'leander',
 'huntsville',
 'alvin',
 'clute',
 'hondo',
 'palestine',
 'fredericksburg',
 'katy',
 'seguin',
 'kerrville',
 'leon+valley',
 'san+angelo',
 'nacogdoches',
 'port+lavaca',
 'orange',
 'natchitoches',
 'college+station',
 'jacksonville',
 'victoria',
 'conroe',
 'bryan',
 'lumberton',
 'belton',
 'baytown',
 'liberty',
 'nederland',
 'tomball',
 'stafford',
 'georgetown',
 'atascocita',
 'lakeway',
 'bay+city',
 'san+marcos',
 'eagle+pass',
 'sulphur',
 'del+rio',
 'south+houston',
 'san+antonio',
 'kirby',
 'texas+city',
 'lake+jackson',
 'robinson',
 'live+oak',
 'jacinto+city',
 'canyon+lake',
 'harker+heights',
 'kyle',
 'vidor',
 'angleton',
 'freeport',
 'pasadena',
 'west+university+place',
 'beeville',
 'bridge+city',
 'mission+bend',
 'woodway',
 'corsicana',
 'missouri+city',
 'b

In [7]:
#set function to get today and previous 4 days' weather data
def unix_time():
    today = datetime.date.today()
    days = [1,2,3,4]
    times = [int(time.mktime(today.timetuple()))]
    for day in days:
        x=today - datetime.timedelta(days=day)
        unixtime = int(time.mktime(x.timetuple()))
        times.append(unixtime)
    return (times)

In [50]:
 base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
temp=[]
wind_speed=[]
pressure=[]
city_name=[]

for city in target_cities[:5]:
    query_url=f"{base_current_url}appid={weather_api_key}&q={city},US-TX,US&units=imperial"
    req = requests.get(query_url)
    response1=req.json()
    latitude=response1["coord"]["lat"]
    longitude=response1["coord"]["lon"]

    for x in unix_time():
        historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}&units=imperial"
        hist_json = requests.get(historical_url).json()
        temp.append(hist_json["current"]["temp"])
        pressure.append(hist_json["current"]["pressure"])
        wind_speed.append(hist_json["current"]["wind_speed"])
        city_name.append(response1['name'])
        print(historical_url)

KeyError: 'current'

In [39]:
#set base url and define function to find OpenWeather cities' based on city name from nearest place API and feed back into
#OpenWeather one call API to ensure proper lat and lng

def find_location(target_cities):
    unix_time()
    base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
    temp=[]
    wind_speed=[]
    pressure=[]
    city_name=[]
    for city in target_cities:
        try: 
            query_url=f"{base_current_url}appid={weather_api_key}&q={city},US-TX,US&units=imperial"
            req = requests.get(query_url)
            response1=req.json()
            latitude=response1["coord"]["lat"]
            longitude=response1["coord"]["lon"]
            
            
        except:
            print("City not found. Skipping...")
            pass
        for x in unix_time():
            historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}&units=imperial"
            hist_json = requests.get(historical_url).json()
            temp.append(hist_json["current"]["temp"])
            pressure.append(hist_json["current"]["pressure"])
            wind_speed.append(hist_json["current"]["wind_speed"])
            city_name.append(response1['name'])
            
    d=dict()
    d["City Name"] = city_name
    d["Temperature"] = temp
    d["Wind_Speed"] = wind_speed
    d["Pressure"] = pressure    
    return(d)
        #print ("--"*30)

In [40]:
find_location(target_cities)

{'City Name': ['Port Arthur',
  'Port Arthur',
  'Port Arthur',
  'Port Arthur',
  'Port Arthur',
  'Brownwood',
  'Brownwood',
  'Brownwood',
  'Brownwood',
  'Brownwood',
  'Wharton',
  'Wharton',
  'Wharton',
  'Wharton',
  'Wharton',
  'El Campo',
  'El Campo',
  'El Campo',
  'El Campo',
  'El Campo',
  'Taylor',
  'Taylor',
  'Taylor',
  'Taylor',
  'Taylor',
  'Pleasanton',
  'Pleasanton',
  'Pleasanton',
  'Pleasanton',
  'Pleasanton',
  'Copperas Cove',
  'Copperas Cove',
  'Copperas Cove',
  'Copperas Cove',
  'Copperas Cove',
  'Lufkin',
  'Lufkin',
  'Lufkin',
  'Lufkin',
  'Lufkin',
  'Temple',
  'Temple',
  'Temple',
  'Temple',
  'Temple',
  'Galveston',
  'Galveston',
  'Galveston',
  'Galveston',
  'Galveston',
  'Uvalde',
  'Uvalde',
  'Uvalde',
  'Uvalde',
  'Uvalde',
  'Lockhart',
  'Lockhart',
  'Lockhart',
  'Lockhart',
  'Lockhart',
  'Brenham',
  'Brenham',
  'Brenham',
  'Brenham',
  'Brenham',
  'Gatesville',
  'Gatesville',
  'Gatesville',
  'Gatesville',
  '

In [41]:
print(historical_url)

NameError: name 'historical_url' is not defined

In [42]:
df =pd.DataFrame(find_location(target_cities))

KeyboardInterrupt: 

In [33]:

df

,City Name,Temperature,Wind_Speed,Pressure
0,Port Arthur,78.60,10.36,1009
1,Port Arthur,76.21,5.75,1014
2,Port Arthur,77.77,4.61,1014
3,Port Arthur,79.34,11.50,1014
4,Port Arthur,79.47,0.00,1016
5,Brownwood,75.56,0.00,1015
6,Brownwood,76.91,4.61,1016
7,Brownwood,77.45,10.36,1016
8,Brownwood,77.04,3.44,1016
9,Brownwood,79.52,3.44,1018
